In [ ]:
# Dự đoán giá cổ phiếu ngày mai dựa trên giá của 10 ngày trước.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Dữ liệu giả lập: Chuỗi giá cổ phiếu
data = np.sin(np.linspace(0, 100, 500))  # Sóng sin giả lập
X = np.array([data[i:i+10] for i in range(len(data)-10)])
y = data[10:]

# Reshape dữ liệu cho LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Xây dựng mô hình LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(10, 1)),
    Dense(1)  # Dự đoán giá trị tiếp theo
])
model.compile(optimizer='adam', loss='mse')

# Huấn luyện mô hình
model.fit(X, y, epochs=10, batch_size=32)

# Dự đoán giá tiếp theo
test_input = data[-10:].reshape(1, 10, 1)
predicted_price = model.predict(test_input)
print(f"Dự đoán giá cổ phiếu ngày mai: {predicted_price[0][0]:.2f}")


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.4551
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3006
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2099
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1295
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0513
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0113
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0039
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0020
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013    
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0010    
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
Dự đoán giá cổ phiếu ngày mai: -0.36


In [4]:
# Phân loại các câu là tích cực hoặc tiêu cực.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Dữ liệu giả lập
texts = ["I love this product", "This is terrible", "Amazing experience", "Not good at all"]
labels = np.array([1, 0, 1, 0])  # 1: Tích cực, 0: Tiêu cực

# Tokenizer và Padding
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=10)  # Đảm bảo tất cả chuỗi có cùng độ dài

# Reshape dữ liệu để phù hợp với LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))  # Thêm một trục để LSTM nhận diện

# Xây dựng mô hình LSTM
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2])),
    Dense(1, activation='sigmoid')  # Phân loại nhị phân
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X, labels, epochs=5, batch_size=2)

# Dự đoán cảm xúc
new_texts = ["I hate this", "This is fantastic"]
new_X = tokenizer.texts_to_sequences(new_texts)
new_X = pad_sequences(new_X, maxlen=10)
new_X = new_X.reshape((new_X.shape[0], new_X.shape[1], 1))  # Reshape để phù hợp với mô hình

predictions = model.predict(new_X)

for i, text in enumerate(new_texts):
    sentiment = "Tích cực" if predictions[i][0] >= 0.5 else "Tiêu cực"
    print(f"Câu: '{text}' -> {sentiment} ({predictions[i][0]:.2f})")


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6667 - loss: 0.6174
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3333 - loss: 0.7537    
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5000 - loss: 0.6875 
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6400 
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3333 - loss: 0.7284     
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Câu: 'I hate this' -> Tiêu cực (0.49)
Câu: 'This is fantastic' -> Tiêu cực (0.46)


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

# Tạo dữ liệu giả lập với quy luật
num_samples = 1000  # Số lượng mẫu dữ liệu
max_sequence_length = 5  # Độ dài tối đa của chuỗi
input_vocab_size = 50  # Số lượng từ hoặc tín hiệu trong đầu vào
output_vocab_size = 5  # Số lượng thông báo đầu ra (5 loại thông báo)

# Quy luật: Nếu giá trị tín hiệu tăng dần, thông báo là "tăng nhiệt độ"
encoder_input_data = []
decoder_input_data = []
decoder_output_data = []

for _ in range(num_samples):
    signal = np.sort(np.random.randint(1, input_vocab_size, max_sequence_length))  # Tín hiệu tăng dần
    encoder_input_data.append(signal)

    if np.all(np.diff(signal) > 0):  # Nếu tín hiệu tăng dần
        message = [1, 1, 1, 0, 0]  # Mã hóa thông báo "tăng nhiệt độ"
    else:
        message = [2, 2, 2, 0, 0]  # Mã hóa thông báo khác

    decoder_input_data.append(message[:-1])
    decoder_output_data.append(message[1:])

# Chuyển dữ liệu thành numpy array
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)
decoder_output_data = np.array(decoder_output_data)

# One-hot encode đầu ra
decoder_output_data_one_hot = np.zeros((num_samples, max_sequence_length - 1, output_vocab_size), dtype="float32")
for i in range(num_samples):
    for t, word in enumerate(decoder_output_data[i]):
        decoder_output_data_one_hot[i, t, word] = 1

# Xây dựng mô hình Seq2Seq
# 1. Encoder
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(input_dim=input_vocab_size, output_dim=128)(encoder_inputs)
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# 2. Decoder
decoder_inputs = Input(shape=(max_sequence_length - 1,))
decoder_embedding = Embedding(input_dim=output_vocab_size, output_dim=128)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Kết hợp Encoder và Decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Huấn luyện mô hình
model.fit([encoder_input_data, decoder_input_data], decoder_output_data_one_hot, batch_size=32, epochs=20)

# Mô hình dự đoán (Inference)
# Encoder Model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Model
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Hàm dịch chuỗi tín hiệu IoT
def decode_sequence(input_seq):
    # Mã hóa chuỗi đầu vào thành trạng thái
    states_value = encoder_model.predict(input_seq)

    # Khởi tạo chuỗi đầu vào cho Decoder
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1  # Bắt đầu với token "start"

    # Dịch chuỗi tín hiệu
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Lấy từ có xác suất cao nhất
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Kiểm tra điều kiện dừng
        if sampled_token_index == 0 or len(decoded_sentence) > max_sequence_length:
            stop_condition = True

        # Cập nhật chuỗi đầu vào
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Cập nhật trạng thái
        states_value = [h, c]

    return decoded_sentence

# Dịch chuỗi tín hiệu mới
test_input_seq = encoder_input_data[0:1]
decoded_sentence = decode_sequence(test_input_seq)

# Hiển thị kết quả
print("Chuỗi tín hiệu cảm biến:", test_input_seq[0])
print("Thông báo dự đoán:", decoded_sentence)



Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5090 - loss: 1.1993
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9833 - loss: 0.0853
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0062
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0027
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0012
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 8.7710e-04
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 7.1128e-04
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 6.0119e-04
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 4.9047e-04
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 4.2066e-04
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1